[View in Colaboratory](https://colab.research.google.com/github/yingjie-wu/Reinforcement-Learning-StarAI/blob/master/FrozenLakeQLearning_Exercise.ipynb)

In [0]:
#           _____                _____                    _____                    _____                    _____                    _____          
#         /\    \              /\    \                  /\    \                  /\    \                  /\    \                  /\    \         
#        /::\    \            /::\    \                /::\    \                /::\    \                /::\    \                /::\    \        
#       /::::\    \           \:::\    \              /::::\    \              /::::\    \              /::::\    \               \:::\    \       
#      /::::::\    \           \:::\    \            /::::::\    \            /::::::\    \            /::::::\    \               \:::\    \      
#     /:::/\:::\    \           \:::\    \          /:::/\:::\    \          /:::/\:::\    \          /:::/\:::\    \               \:::\    \     
#    /:::/__\:::\    \           \:::\    \        /:::/__\:::\    \        /:::/__\:::\    \        /:::/__\:::\    \               \:::\    \    
#    \:::\   \:::\    \          /::::\    \      /::::\   \:::\    \      /::::\   \:::\    \      /::::\   \:::\    \              /::::\    \   
#  ___\:::\   \:::\    \        /::::::\    \    /::::::\   \:::\    \    /::::::\   \:::\    \    /::::::\   \:::\    \    ____    /::::::\    \  
# /\   \:::\   \:::\    \      /:::/\:::\    \  /:::/\:::\   \:::\    \  /:::/\:::\   \:::\____\  /:::/\:::\   \:::\    \  /\   \  /:::/\:::\    \ 
#/::\   \:::\   \:::\____\    /:::/  \:::\____\/:::/  \:::\   \:::\____\/:::/  \:::\   \:::|    |/:::/  \:::\   \:::\____\/::\   \/:::/  \:::\____\
#\:::\   \:::\   \::/    /   /:::/    \::/    /\::/    \:::\  /:::/    /\::/   |::::\  /:::|____|\::/    \:::\  /:::/    /\:::\  /:::/    \::/    /
# \:::\   \:::\   \/____/   /:::/    / \/____/  \/____/ \:::\/:::/    /  \/____|:::::\/:::/    /  \/____/ \:::\/:::/    /  \:::\/:::/    / \/____/ 
#  \:::\   \:::\    \      /:::/    /                    \::::::/    /         |:::::::::/    /            \::::::/    /    \::::::/    /          
#   \:::\   \:::\____\    /:::/    /                      \::::/    /          |::|\::::/    /              \::::/    /      \::::/____/           
#    \:::\  /:::/    /    \::/    /                       /:::/    /           |::| \::/____/               /:::/    /        \:::\    \           
#     \:::\/:::/    /      \/____/                       /:::/    /            |::|  ~|                    /:::/    /          \:::\    \          
#      \::::::/    /                                    /:::/    /             |::|   |                   /:::/    /            \:::\    \         
#       \::::/    /                                    /:::/    /              \::|   |                  /:::/    /              \:::\____\        
#        \::/    /                                     \::/    /                \:|   |                  \::/    /                \::/    /        
#         \/____/                                       \/____/                  \|___|                   \/____/                  \/____/         

In [0]:
!pip install gym > /dev/null 2>&1

import gym
import numpy as np
import random

## Tabular Q - Simplified

## $$ Q(S_{t}, A_{t}) = R_{t+1} + \gamma \ \underset{a}{\operatorname{max}} Q(S_{t+1}, a) $$
$ Q(S_{t}, A_{t}) \rightarrow $ A function that returns Q value given params (S, A)  
$ R_{t+1} \rightarrow $ Reward of next state  
$ \gamma \rightarrow $ discount rate  
$ \underset{a}{\operatorname{max}} Q(S_{t+1}, a) \rightarrow $ Best Q value of next state


## Tabular Q - Enhanced for random transitions
This formula works for deterministic transitions, i.e. if I move left I always end up in the state to my left. But in the real world we can't build perfect models and sensors for our robots will have a slight error. The environment also may not react to the same action the same way all the time so this simplified formula doesn't quite work because it updates with the entirety of the next state we saw when in reality there could be multiple next states (e.g. 30% state 1, 20% state 2, 50% state 3)
## $$ Q(S_{t}, A_{t}) \leftarrow R_{t+1} + \gamma \ \underset{a}{\operatorname{max}} Q(S_{t+1}, a) $$  
### When the left and right doesn't match
If we look at it from the perspective of how wrong our current Q is to what we just experienced, i.e. the difference between the left and right side of the above formula. The below would be considered how wrong our knowledge is compared to what we just experienced.
## $$ TD\ Error = [R_{t+1} + \gamma \ \underset{a}{\operatorname{max}} Q(S_{t+1}, a)] -  Q(S_{t}, A_{t})$$  
### An enhanced learning process
So if we update our Q with only a small percentage of the error in our most recent experience, over many visits we will converge towards a value that is representative of the expected value of Q across multiple possible next states. 
## $$ Q(S_{t}, A_{t}) \leftarrow Q(S_{t}, A_{t}) + \alpha [Error] $$  
### The final formula
## $$ Q(S_{t}, A_{t}) \leftarrow Q(S_{t}, A_{t}) + \alpha [R_{t+1} + \gamma \ \underset{a}{\operatorname{max}} Q(S_{t+1}, a) - Q(S_{t}, A_{t})] $$  
$ \ $  
$ Q(S_{t}, A_{t}) \rightarrow $ A function that returns Q value given params (S, A)  
$ \alpha \rightarrow $ the learning rate, i.e. it adjusts how much of the new experience we store into $Q(S_{t}, A_{t})$  
$ R_{t+1} \rightarrow $ Reward of next state  
$ \gamma \rightarrow $ the discount rate, how much we discount future reward per time step  
$ \underset{a}{\operatorname{max}} Q(S_{t+1}, a) \rightarrow $ Best Q value of next state


# Linear Frozen Lake environment

In [0]:
"""NO NEED TO CHANGE THIS CELL"""
import sys
from six import StringIO, b

from gym import utils
from gym.envs.toy_text import discrete

LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3

MAPS = {
    "4x4": [
        "SFFF",
        "FHFH",
        "FFFH",
        "HFFG"
    ],
    "1x8": [
        "HFFSFFFG"
    ],
    "8x8": [
        "SFFFFFFF",
        "FFFFFFFF",
        "FFFHFFFF",
        "FFFFFHFF",
        "FFFHFFFF",
        "FHHFFFHF",
        "FHFFHFHF",
        "FFFHFFFG"
    ],
}

class FrozenLakeEnv(discrete.DiscreteEnv):
    """
    Winter is here. You and your friends were tossing around a frisbee at the park
    when you made a wild throw that left the frisbee out in the middle of the lake.
    The water is mostly frozen, but there are a few holes where the ice has melted.
    If you step into one of those holes, you'll fall into the freezing water.
    At this time, there's an international frisbee shortage, so it's absolutely imperative that
    you navigate across the lake and retrieve the disc.
    However, the ice is slippery, so you won't always move in the direction you intend.
    The surface is described using a grid like the following
        SFFF
        FHFH
        FFFH
        HFFG
    S : starting point, safe
    F : frozen surface, safe
    H : hole, fall to your doom
    G : goal, where the frisbee is located
    The episode ends when you reach the goal or fall in a hole.
    You receive a reward of 1 if you reach the goal, and zero otherwise.
    """

    metadata = {'render.modes': ['human', 'ansi']}

    def __init__(self, desc=None, map_name="4x4",is_slippery=True):
        if desc is None and map_name is None:
            raise ValueError('Must provide either desc or map_name')
        elif desc is None:
            desc = MAPS[map_name]
        self.desc = desc = np.asarray(desc,dtype='c')
        self.nrow, self.ncol = nrow, ncol = desc.shape
        self.reward_range = (0, 1)

        nA = 4
        nS = nrow * ncol

        isd = np.array(desc == b'S').astype('float64').ravel()
        isd /= isd.sum()

        P = {s : {a : [] for a in range(nA)} for s in range(nS)}

        def to_s(row, col):
            return row*ncol + col
        
        def inc(row, col, a):
            if a==0: # left
                col = max(col-1,0)
            elif a==1: # down
                row = min(row+1,nrow-1)
            elif a==2: # right
                col = min(col+1,ncol-1)
            elif a==3: # up
                row = max(row-1,0)
            return (row, col)

        for row in range(nrow):
            for col in range(ncol):
                s = to_s(row, col)
                for a in range(4):
                    li = P[s][a]
                    letter = desc[row, col]
                    if letter in b'GH':
                        li.append((1.0, s, 0, True))
                    else:
                        if is_slippery:
                            for b in [(a-1)%4, a, (a+1)%4]:
                                newrow, newcol = inc(row, col, b)
                                newstate = to_s(newrow, newcol)
                                newletter = desc[newrow, newcol]
                                done = bytes(newletter) in b'GH'
                                rew = float(newletter == b'G')
                                li.append((1.0/3.0, newstate, rew, done))
                        else:
                            newrow, newcol = inc(row, col, a)
                            newstate = to_s(newrow, newcol)
                            newletter = desc[newrow, newcol]
                            done = bytes(newletter) in b'GH'
                            rew = float(newletter == b'G')
                            li.append((1.0, newstate, rew, done))

        super(FrozenLakeEnv, self).__init__(nS, nA, P, isd)

    def render(self, mode='human'):
        outfile = StringIO() if mode == 'ansi' else sys.stdout

        row, col = self.s // self.ncol, self.s % self.ncol
        desc = self.desc.tolist()
        desc = [[c.decode('utf-8') for c in line] for line in desc]
        desc[row][col] = utils.colorize(desc[row][col], "red", highlight=True)
        if self.lastaction is not None:
            outfile.write("  ({})\n".format(["Left","Down","Right","Up"][self.lastaction]))
        else:
            outfile.write("\n")
        outfile.write("\n".join(''.join(line) for line in desc)+"\n")

        if mode != 'human':
            return outfile

        


# Environment Description

In [3]:
"""
Winter is here. You and your friends were tossing around a frisbee at the park
when you made a wild throw that left the frisbee out in the middle of the lake.
The water is mostly frozen, but there are a few holes where the ice has melted.
If you step into one of those holes, you'll fall into the freezing water.
At this time, there's an international frisbee shortage, so it's absolutely imperative that
you navigate across the lake and retrieve the disc.
However, the ice is slippery, so you won't always move in the direction you intend.
The surface is described using a grid like the following
    SFFF
    FHFH
    FFFH
    HFFG
S : starting point, safe
F : frozen surface, safe
H : hole, fall to your doom
G : goal, where the frisbee is located
The episode ends when you reach the goal or fall in a hole.
You receive a reward of 1 if you reach the goal, and zero otherwise.

Action space:
LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3
"""

"\nWinter is here. You and your friends were tossing around a frisbee at the park\nwhen you made a wild throw that left the frisbee out in the middle of the lake.\nThe water is mostly frozen, but there are a few holes where the ice has melted.\nIf you step into one of those holes, you'll fall into the freezing water.\nAt this time, there's an international frisbee shortage, so it's absolutely imperative that\nyou navigate across the lake and retrieve the disc.\nHowever, the ice is slippery, so you won't always move in the direction you intend.\nThe surface is described using a grid like the following\n    SFFF\n    FHFH\n    FFFH\n    HFFG\nS : starting point, safe\nF : frozen surface, safe\nH : hole, fall to your doom\nG : goal, where the frisbee is located\nThe episode ends when you reach the goal or fall in a hole.\nYou receive a reward of 1 if you reach the goal, and zero otherwise.\n\nAction space:\nLEFT = 0\nDOWN = 1\nRIGHT = 2\nUP = 3\n"

In [4]:
env = FrozenLakeEnv(map_name="1x8",is_slippery=False)
print("Environment state:")
env.render()
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)
env.close()

Environment state:

HFFSFFFG
Observation space: Discrete(8)
Action space: Discrete(4)


# Scripted Interaction

In [5]:
observation = env.reset()

env.render()
"""walk left"""
print("walking left")
action = 0
env.step(action)
env.render()

"""walk right 3x"""
print("walking right 3x")
action = 2 
env.step(action)
env.render()
env.step(action)
env.render()
env.step(action)
env.render()

"""walk down  (nothing should happen, same with up)"""
env.step(1)
print("walking down")
env.render()

"""walk right again"""
print("walking right 2x")
observation, reward, done, info = env.step(2)
print("Observation:", observation, "Reward:", reward, "Done:", done)
env.render()

print("walking right 2x")
observation, reward, done, info = env.step(2)
env.render()
print("Observation:", observation, "Reward:", reward, "Done:", done)

env.close()


HFFSFFFG
walking left
  (Left)
HFFSFFFG
walking right 3x
  (Right)
HFFSFFFG
  (Right)
HFFSFFFG
  (Right)
HFFSFFFG
walking down
  (Down)
HFFSFFFG
walking right 2x
Observation: 6 Reward: 0.0 Done: False
  (Right)
HFFSFFFG
walking right 2x
  (Right)
HFFSFFFG
Observation: 7 Reward: 1.0 Done: True


# Random Interaction
Run the cell below a few times and you'll notice that a random agent can take between a few steps to over 100 steps to navigate this straight line maze.

Keeping in mind this is just limiting the actions to left or right

In [43]:
"""Run for 1 episode"""
for i_episode in range(1):
    observation = env.reset()
    for t in range(500):
        env.render()
        action = random.sample([0, 2], 1)[0] #pick 1 sample from 0 (left) or 2 (right)
        observation, reward, done, info = env.step(action)
        if done:
            env.render()
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()


HFFSFFFG
  (Left)
HFFSFFFG
  (Right)
HFFSFFFG
  (Right)
HFFSFFFG
  (Right)
HFFSFFFG
  (Left)
HFFSFFFG
  (Right)
HFFSFFFG
  (Right)
HFFSFFFG
  (Right)
HFFSFFFG
Episode finished after 8 timesteps


# The tools

## Epsilon ($\varepsilon$) Greedy - explore or exploit
The greek letter epsilon is used to indicate exploration rate, it's the probability that our agent will explore.

$\varepsilon$ = 1.00 $\to$ 100% explore  
$\varepsilon$ = 0.00 $\to$ 100% exploit  

Here we'll create a class that makes it easy for us to keep track of our epsilon value, with a flag (epsilon.isTraining) to indicate whether we're in training mode or in run mode.

There are many approaches to epsilon-greedy, this is just one way

In [7]:
class Epsilon(object):
    def __init__(self, start=1.0, end=0.01, update_decrement=0.01):
        self.start = start
        self.end = end
        self.update_decrement = update_decrement
        self._value = self.start
        self.isTraining = True
    
    def decrement(self, count=1):
        self._value = max(self.end, self._value - self.update_decrement*count)
        return self
        
    def value(self):
        if not self.isTraining:
            return 0.0
        else:
            return self._value
"""
Instantiate object with epsilon starting at 1.0 (100% exploration), final value 0.01 (1% exploration), 
each time we call decrement it'll go down by 0.01. 
If eps.isTraining is set to True then it'll return 0.0 (zero exploration)
"""
eps = Epsilon(start=1.0, end=0.01, update_decrement=0.01)
print(eps.value())
print("decrementing 3 times")
print(eps.decrement().value())
print(eps.decrement().value())
print(eps.decrement().value())
print("decrement 99 times and the lowest it goes to is 0.01")
print(eps.decrement(99).value())
print("Set training = False")
eps.isTraining = False
print(eps.decrement().value())
print("Set training = True")
eps.isTraining = True
print(eps.decrement().value())

1.0
decrementing 3 times
0.99
0.98
0.97
decrement 99 times and the lowest it goes to is 0.01
0.01
Set training = False
0.0
Set training = True
0.01



## Implementing $Q(S_{t}, A_{t})$
This can implemented as a table where the index is the state, each record is a list of the Q values for that state which in our scenario is 2 Q values (move left and right)


| State         | Left (0)      | Right (2) |
| ------------- | -------------:| ---------:|
| state 1       |          0.45 |      0.87 |
| state 2       |          0.35 |      0.54 |
| state 3       |          0.73 |      0.34 |

The python dictionary is a good mechanism for this

In [8]:
"""Create a dictionary as our Q table"""
Q = {}

"""Insert a single state action pair"""
print("Simple State")
s = tuple([5])
a = [90, 92]  # left and right Q values
Q[s] = a

print("s:", s)
print("Retrieving (s):", Q[s])
print("Retrieving (s, left):", Q[s][0])
print("Retrieving (s, right):", Q[s][1], "\n")

"""Insert a complex state"""
print("Complex state")
s = tuple([5, 6, 8, 9])
a = [100, 102]  # left and right Q values
Q[s] = a

print("s:", s)
print("Retrieving (s):", Q[s])
print("Retrieving (s, left):", Q[s][0])
print("Retrieving (s, right):", Q[s][1], "\n")

"""Overriding a Q value"""
print("Overriding a Q value")
print("Q before overwriting:", Q)
Q[s][0] = 101
print("Q after overwriting:", Q)
print("Retrieving (s):", Q[s])

Simple State
s: (5,)
Retrieving (s): [90, 92]
Retrieving (s, left): 90
Retrieving (s, right): 92 

Complex state
s: (5, 6, 8, 9)
Retrieving (s): [100, 102]
Retrieving (s, left): 100
Retrieving (s, right): 102 

Overriding a Q value
Q before overwriting: {(5,): [90, 92], (5, 6, 8, 9): [100, 102]}
Q after overwriting: {(5,): [90, 92], (5, 6, 8, 9): [101, 102]}
Retrieving (s): [101, 102]


# Exercise - Implement a Q Table that can store 4 actions
Let's implement a class that will take care of our Q table operations for us. This way our code for the agent can be easier to read and organize. 

e.g. we can call Q.get_Q(s, a) which will take care of initialization checks without the agent having to worrying about it

Also comes with a print() to make it easy to see what our q table looks like

#### Methods:  
get_Q(s, a): This method will return the Q value of (s, a) pair    $\rightarrow Q(s, a)$  
get_max_Q(s): This method will get the max of all Q values given state s  $\rightarrow \underset{a}{\operatorname{max}} Q(s, a)$  
set_Q(s, a, q): This method will assign a new q value to Q(s, a)  $\rightarrow Q(s, a) = q$  
get_max_a_for_Q(s): This method will return the action that maximises Q, so we can take the greedy action  $\rightarrow \underset{a}{\operatorname{argmax}} Q(s, a)$  


In [0]:
class QTable():
    def __init__(self, num_actions=4):
        self.num_actions = num_actions
        self.Q = {}
    
    """Q(s, a): get the Q value of (s, a) pair"""
    def get_Q(self, s, a):
        self._check(s)
        return self.Q[s][a]

    """if Q(s) doesn't exist initialize to 0"""
    def _check(self, s):
        if not s in self.Q:
            self.Q[s] = [0.]*self.num_actions
    
    """max Q(s): get the max of all Q value of state s"""
    def get_max_Q(self, s):
        self._check(s)
        return np.max(self.Q[s])
    
    """Q(s, a) = q: update the q value of (s, a) pair"""
    def set_Q(self, s, a, q):
        self._check(s)
        self.Q[s][a] = q
    
    """
    argmax_a Q(s, a): get the action which has the highest Q in state s
    - beware of the scenario where all Q[s] are equal, np.argmax just returns the first one
    """
    def get_max_a_for_Q(self, s):
        self._check(s)
        if np.min(self.Q[s]) == np.max(self.Q[s]):
          return random.choice([i for i in range(self.num_actions)]) # if all values are equal then do a random choice
        else:
          return np.argmax(self.Q[s])
    
    def __str__(self):
        output = []
        for s in self.Q:
            output.append(s.__str__() + ": " + ["{:07.4f}".format(a) for a in self.Q[s]].__str__())
        output.sort()
        return "QTable (number of actions = " + str(self.num_actions) + ", states = " + str(len(output)) + "):\n" + "\n".join(output)

In [49]:
"""Tests"""

Q = QTable(num_actions=4)

s = tuple([5, 6])
a = 1

assert Q.get_Q(s, a) == 0, "Q value should be 0 to start with"

s = tuple([5, 3])
Q.set_Q(s, a, 90)
assert Q.get_Q(s, a) == 90, "Updated Q value should equal 90"

a = 2
Q.set_Q(s, a, 85)
assert Q.get_max_Q(s) == 90, "Max Q should be 90"
assert Q.get_max_a_for_Q(s) == 1, "Max action for state should be 1"


print(Q)

QTable (number of actions = 4, states = 2):
(5, 3): ['00.0000', '90.0000', '85.0000', '00.0000']
(5, 6): ['00.0000', '00.0000', '00.0000', '00.0000']


# Exercise - Make an agent that runs the frozen lake

### Pseudocode for Q Learning
```
Initialize Q(s, a), for all S, A, arbitrarily  (can do delayed initialisation)  
Repeat (for each episode)  
    Initialize S  
    Repeat (for each step of episode):  
        Choose A from S using policy derived from Q (epsilon - Greedy)  
        Take action, observe R, S'
        Q(S, A) <-- Q(S,A) + alpha [R + gamma * maxQ(S', a) - Q(S,A)]
        S <-- S'
    until S is terminal
```
  
note, can't do math notation in a code block so:  
*S' is $S_{t+1}$*  
*the maxQ(S', a) above is $\underset{a}{\operatorname{max}} Q(S_{t+1}, a)$*

### Suggested progression
- Get agent training loop working with random action
- Get agent action selection to use epsilon-greedy
- Get agent to store and update Q values

In [36]:
env.observation_space.sample() 

3

In [0]:
class Agent():
    def __init__(self):
        self.env = FrozenLakeEnv(map_name="1x8",is_slippery=False)
        self.Q = QTable(num_actions=4)
        self.epsilon = Epsilon(start=1.0, end=0.01, update_decrement=0.01)
    
    def getAction(self, s):
        if np.random.uniform() > self.epsilon.value():
          ## expoitation mode
          action = self.Q.get_max_a_for_Q(s)
        else:
          ## exploration mode
          action = self.env.action_space.sample()
        return action
    
    def train(self, episodes=20, alpha=0.1, gamma=0.9):
        # TODO: 
        # - get training loop working with random action
        # - store and update Q values
        self.epsilon.isTraining = True      
        
        for i in range(episodes):
          #print("Q:", self.Q)                   
          s = self.env.reset()
          s = tuple([s])
          #print(s)
          steps = 0
          while True and steps <= 30:
              #self.env.render()
              # take action and have observation
              action = self.getAction(s)
              s_1, reward, done, info = self.env.step(action)
              
              ## update Q value
              s_1 = tuple([s_1])
              Q_new = self.Q.get_Q(s, action) + alpha * (reward + gamma * (self.Q.get_max_Q(s_1)) - self.Q.get_Q(s, action))
              self.Q.set_Q(s, action, Q_new)
              #print("s:", s)
              #print("action:", action)
              #print("reward:", reward)
              #print("Q_new:", Q_new)
              #print("Q (after set):", self.Q) 
              
              ## move to next stage
              s = s_1
              steps += 1
              if done:
                  print("Episode finished after {} timesteps".format(steps))
                  self.epsilon.decrement().value()
                  break        
                
    
    def run(self):
        print("Running agent with this Q table")
        print("Q:", self.Q)
        self.epsilon.isTraining = False
        s = self.env.reset()
        s = tuple([s])
        print(s)
        steps = 0
        while True:
            self.env.render()
            action = self.getAction(s)
            s_1, reward, done, info = self.env.step(action)
            s_1 = tuple([s_1])
            s = s_1
            steps += 1
            if done:
                print("Episode finished after {} timesteps".format(steps))
                break
              

In [90]:
env = FrozenLakeEnv(map_name="1x8",is_slippery=False)
agent = Agent()
agent.env = env
agent.train(episodes=100, alpha=0.1, gamma=0.9)
agent.run()

Episode finished after 22 timesteps
Episode finished after 29 timesteps
Episode finished after 30 timesteps
Episode finished after 5 timesteps
Episode finished after 7 timesteps
Episode finished after 12 timesteps
Episode finished after 21 timesteps
Episode finished after 24 timesteps
Episode finished after 28 timesteps
Episode finished after 7 timesteps
Episode finished after 28 timesteps
Episode finished after 6 timesteps
Episode finished after 5 timesteps
Episode finished after 20 timesteps
Episode finished after 25 timesteps
Episode finished after 25 timesteps
Episode finished after 20 timesteps
Episode finished after 15 timesteps
Episode finished after 6 timesteps
Episode finished after 14 timesteps
Episode finished after 16 timesteps
Episode finished after 7 timesteps
Episode finished after 9 timesteps
Episode finished after 15 timesteps
Episode finished after 20 timesteps
Episode finished after 5 timesteps
Episode finished after 14 timesteps
Episode finished after 7 timesteps
Ep

# Exercise - Try the agent on a more complicated frozen lake environment

In [100]:
env = FrozenLakeEnv(map_name="4x4",is_slippery=False)
agent = Agent()
agent.env = env
agent.train(episodes=100, alpha=0.1, gamma=0.9) # What is the good number of episodes to use? What if is_slipper is False?

Episode finished after 6 timesteps
Episode finished after 25 timesteps
Episode finished after 4 timesteps
Episode finished after 4 timesteps
Episode finished after 2 timesteps
Episode finished after 2 timesteps
Episode finished after 18 timesteps
Episode finished after 4 timesteps
Episode finished after 4 timesteps
Episode finished after 3 timesteps
Episode finished after 5 timesteps
Episode finished after 21 timesteps
Episode finished after 15 timesteps
Episode finished after 4 timesteps
Episode finished after 4 timesteps
Episode finished after 3 timesteps
Episode finished after 15 timesteps
Episode finished after 13 timesteps
Episode finished after 15 timesteps
Episode finished after 9 timesteps
Episode finished after 4 timesteps
Episode finished after 11 timesteps
Episode finished after 2 timesteps
Episode finished after 10 timesteps
Episode finished after 3 timesteps
Episode finished after 7 timesteps
Episode finished after 10 timesteps
Episode finished after 11 timesteps
Episode f

In [101]:
agent.run()

Running agent with this Q table
Q: QTable (number of actions = 4, states = 16):
(0,): ['00.0000', '00.0000', '00.0018', '00.0000']
(1,): ['00.0000', '00.0000', '00.0108', '00.0000']
(10,): ['00.0000', '00.4100', '00.0000', '00.0001']
(11,): ['00.0000', '00.0000', '00.0000', '00.0000']
(12,): ['00.0000', '00.0000', '00.0000', '00.0000']
(13,): ['00.0000', '00.0000', '00.0000', '00.0000']
(14,): ['00.0000', '00.0000', '00.7941', '00.0023']
(15,): ['00.0000', '00.0000', '00.0000', '00.0000']
(2,): ['00.0000', '00.0476', '00.0000', '00.0015']
(3,): ['00.0007', '00.0000', '00.0000', '00.0000']
(4,): ['00.0000', '00.0000', '00.0000', '00.0000']
(5,): ['00.0000', '00.0000', '00.0000', '00.0000']
(6,): ['00.0000', '00.1546', '00.0000', '00.0019']
(7,): ['00.0000', '00.0000', '00.0000', '00.0000']
(8,): ['00.0000', '00.0000', '00.0000', '00.0000']
(9,): ['00.0000', '00.0000', '00.0000', '00.0000']
(0,)

SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFF

In [150]:
env = FrozenLakeEnv(map_name="8x8",is_slippery=False)
agent = Agent()
agent.env = env
agent.train(episodes=5000, alpha=0.5, gamma=0.95) # What is the good number of episodes to use? What if is_slipper is False?

Episode finished after 10 timesteps
Episode finished after 13 timesteps
Episode finished after 30 timesteps
Episode finished after 16 timesteps
Episode finished after 26 timesteps
Episode finished after 26 timesteps
Episode finished after 31 timesteps
Episode finished after 11 timesteps
Episode finished after 20 timesteps
Episode finished after 10 timesteps
Episode finished after 21 timesteps
Episode finished after 14 timesteps
Episode finished after 26 timesteps
Episode finished after 26 timesteps
Episode finished after 23 timesteps
Episode finished after 20 timesteps
Episode finished after 25 timesteps
Episode finished after 12 timesteps
Episode finished after 11 timesteps
Episode finished after 8 timesteps
Episode finished after 23 timesteps
Episode finished after 29 timesteps
Episode finished after 29 timesteps
Episode finished after 17 timesteps
Episode finished after 5 timesteps
Episode finished after 31 timesteps
Episode finished after 18 timesteps
Episode finished after 14 time

In [151]:
agent.run()

Running agent with this Q table
Q: QTable (number of actions = 4, states = 63):
(0,): ['00.4572', '00.1448', '00.5133', '00.4572']
(1,): ['00.4839', '00.4153', '00.5404', '00.5093']
(10,): ['00.0000', '00.0000', '00.0000', '00.5361']
(11,): ['00.2487', '00.0000', '00.6302', '00.5677']
(12,): ['00.5613', '00.5309', '00.6634', '00.4896']
(13,): ['00.6204', '00.6983', '00.6824', '00.4900']
(14,): ['00.0000', '00.7347', '00.0000', '00.0000']
(15,): ['00.4267', '00.0000', '00.0000', '00.0000']
(16,): ['00.0000', '00.0000', '00.0000', '00.0000']
(17,): ['00.0000', '00.0000', '00.0000', '00.0000']
(18,): ['00.0000', '00.0000', '00.0000', '00.0000']
(19,): ['00.0000', '00.0000', '00.0000', '00.0000']
(2,): ['00.5053', '00.4612', '00.5688', '00.5361']
(20,): ['00.0000', '00.0000', '00.0000', '00.6290']
(21,): ['00.5391', '00.0000', '00.7351', '00.4976']
(22,): ['00.6874', '00.5553', '00.7738', '00.4810']
(23,): ['00.7121', '00.8145', '00.5756', '00.0839']
(24,): ['00.0000', '00.0000', '00.0000'